# 1. Import Libs

In [1]:
import pandas as pd
import numpy as np
import jsonschema
import json

# 2. Read Json File


In [2]:
# Specify the path to your JSON file
json_file_path = 'sales.json'

# Read the JSON data into a Pandas DataFrame
df = pd.read_json(json_file_path,lines=True)

# Display the DataFrame
df

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
1,{'$oid': '5bd761dcae323e45a93ccff2'},{'$date': '2015-07-25T07:20:29.804Z'},"[{'name': 'pens', 'tags': ['writing', 'office'...",Seattle,"{'gender': 'F', 'age': 34, 'email': 'keigutip@...",False,Phone
2,{'$oid': '5bd761dcae323e45a93ccfee'},{'$date': '2014-11-11T02:13:51.893Z'},"[{'name': 'laptop', 'tags': ['electronics', 's...",London,"{'gender': 'F', 'age': 40, 'email': 'pan@cak.z...",False,In store
3,{'$oid': '5bd761dcae323e45a93ccffa'},{'$date': '2016-08-15T04:05:03.298Z'},"[{'name': 'laptop', 'tags': ['electronics', 's...",San Diego,"{'gender': 'F', 'age': 40, 'email': 'elusekjiv...",True,In store
4,{'$oid': '5bd761dcae323e45a93cd002'},{'$date': '2015-04-18T03:33:09.638Z'},"[{'name': 'laptop', 'tags': ['electronics', 's...",Austin,"{'gender': 'F', 'age': 37, 'email': 'oh@ocdumv...",False,Online
...,...,...,...,...,...,...,...
4995,{'$oid': '5bd761deae323e45a93ce311'},{'$date': '2013-09-27T01:38:17.324Z'},"[{'name': 'pens', 'tags': ['writing', 'office'...",London,"{'gender': 'F', 'age': 32, 'email': 'avcema@re...",False,In store
4996,{'$oid': '5bd761deae323e45a93ce326'},{'$date': '2013-11-03T18:07:08.503Z'},"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 36, 'email': 'ucouwibel...",False,In store
4997,{'$oid': '5bd761deae323e45a93ce346'},{'$date': '2017-10-17T09:33:12.67Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Denver,"{'gender': 'M', 'age': 25, 'email': 'kopecatid...",False,Phone
4998,{'$oid': '5bd761deae323e45a93ce360'},{'$date': '2014-10-08T12:57:07.902Z'},"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 55, 'email': 'du@tieca....",False,Online


In [3]:
# Display column names and data types
for column, dtype in zip(df.columns, df.dtypes):
    print(f"Column: {column}, Data Type: {dtype}")


Column: _id, Data Type: object
Column: saleDate, Data Type: object
Column: items, Data Type: object
Column: storeLocation, Data Type: object
Column: customer, Data Type: object
Column: couponUsed, Data Type: bool
Column: purchaseMethod, Data Type: object


In [4]:
# Define a function to calculate sales for each item
def calculate_sales(row):
    return int(row['quantity']) * float(row['price']['$numberDecimal'])

# Apply the calculate_sales function to the 'items' column to create a new column 'sales'
df['sales'] = df['items'].apply(lambda x: sum(calculate_sales(item) for item in x))

# Create a list to store item names and corresponding sales
item_sales = []

for _, row in df.iterrows():
    for item in row['items']:
        item_name = item['name']
        item_sale = calculate_sales(item)
        item_sales.append((item_name, item_sale))

# Create a DataFrame from the list
item_sales_df = pd.DataFrame(item_sales, columns=['Item Name', 'Sales'])

# Group by item name and sum the sales for each item
item_sales_summary = item_sales_df.groupby('Item Name')['Sales'].sum().reset_index()

# Sort the summary DataFrame by sales in descending order
item_sales_summary = item_sales_summary.sort_values(by='Sales', ascending=False)

# Display the top 10 products by sales
top_10_products = item_sales_summary.head(10)

# Display the top 10 products by sales
top_10_products


,Item Name,Sales
3,laptop,6775977.07
0,backpack,817374.10
5,pens,581843.27
1,binder,511644.57
4,notepad,463615.48
2,envelopes,376658.49
6,printer paper,367459.29


In [5]:
# Create a list to store product names, sales, and store locations
product_sales_location = []

for _, row in df.iterrows():
    for item in row['items']:
        item_name = item['name']
        item_sale = calculate_sales(item)
        location = row['storeLocation']
        product_sales_location.append((location, item_name, item_sale))

# Create a DataFrame from the list
product_sales_location_df = pd.DataFrame(product_sales_location, columns=['Store Location', 'Item Name', 'Sales'])

# Group by store location and item name, and sum the sales for each item at each location
product_sales_summary = product_sales_location_df.groupby(['Store Location', 'Item Name'])['Sales'].sum().reset_index()

# Sort the summary DataFrame by store location and sales in descending order
product_sales_summary = product_sales_summary.sort_values(by=['Store Location', 'Sales'], ascending=[True, False])

# Display the top 3 products by name and sales by store location
top_3_products_by_location = product_sales_summary.groupby('Store Location').head(3)

# Display the top 3 products by name and sales by store location
top_3_products_by_location


,Store Location,Item Name,Sales
3,Austin,laptop,1018494.05
0,Austin,backpack,115217.59
5,Austin,pens,74622.52
10,Denver,laptop,1961659.25
7,Denver,backpack,245679.34
12,Denver,pens,185008.39
17,London,laptop,1073098.72
14,London,backpack,144529.14
19,London,pens,92602.30
24,New York,laptop,694613.32


In [6]:
# Extract the 'gender' attribute from the 'customer' column
df['gender'] = df['customer'].apply(lambda x: x['gender'])

# Create a cross-tabulation of purchase method by gender
purchase_by_gender = pd.crosstab(df['gender'], df['purchaseMethod'], margins=True, margins_name="Total")

# Rename the margins column
purchase_by_gender.rename(columns={'Total': 'Purchase method'}, inplace=True)
purchase_by_gender.rename(index={'Total': 'Gender'}, inplace=True)

# Create a cross-tabulation of purchase method by gender
# purchase_by_gender = pd.crosstab(df['gender'], df['purchaseMethod'])



# Display the formatted purchase by gender table
purchase_by_gender


purchaseMethod,In store,Online,Phone,Purchase method
gender,,,,
F,1430,813,284,2527
M,1389,772,312,2473
Gender,2819,1585,596,5000


In [7]:
# Extract the "$date" values from the "saleDate" dictionaries and convert them to datetime
df['saleDate'] = pd.to_datetime(df['saleDate'].apply(lambda x: x["$date"]))

# Extract the year and month into separate columns
df['Year'] = df['saleDate'].dt.year
df['Month'] = df['saleDate'].dt.month

month_names = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

# Replace month numbers with month names
df['Month'] = df['Month'].map(month_names)
df['Month'] = pd.Categorical(df['Month'], categories=month_names.values(), ordered=True)

# Group the data by year and month and calculate the total sales
monthly_sales = df.groupby(['Year', 'Month'])['sales'].sum()

# Display the monthly total sales
print(monthly_sales)


Year  Month    
2013  January      196043.76
      February     147622.59
      March        140700.06
      April        162336.67
      May          179143.37
      June         142132.02
      July         176298.23
      August       130977.22
      September    137871.99
      October      173126.73
      November     148358.38
      December     174306.96
2014  January      125824.02
      February     141174.41
      March        193413.50
      April        132593.08
      May          166896.80
      June         143304.56
      July         184804.60
      August       167979.81
      September    159802.92
      October      149284.84
      November     180564.85
      December     139466.83
2015  January      222577.01
      February     189776.15
      March        178086.94
      April        132298.50
      May          167482.69
      June         149484.34
      July         149510.47
      August       154868.63
      September    135615.80
      October      175215.3